In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation

Using TensorFlow backend.


In [2]:
import numpy as np
import os

np.random.seed(42)

DATA_PATH = 'dataset'

In [3]:
from scipy.io import arff
import pandas as pd

def load_pulsar_csv(path = DATA_PATH):
    csv_path = os.path.join(path, 'HTRU_2.csv')
    return np.loadtxt(csv_path, delimiter=',', dtype=np.float32)

def load_pulsar_arff(path = DATA_PATH):
    arff_path = os.path.join(path, 'HTRU_2.arff')
    return arff.loadarff(arff_path)

In [4]:
pulsars = load_pulsar_csv()

In [5]:
import numpy as np

def split_train_dataset(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(test_ratio * len(data))
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data[train_indices,:], data[test_indices,:]

# Use hash of identifier to decide if instance goes into train or test set

In [6]:
# Save Model
import os

MODELS_DIR = "models"

def save_model(model, name, acc=None):
    name += str(model.input.shape[1])
    for layer in model.layers:
        name += "-" + str(layer.output.shape[1])
    
    name += "_" + (("%.2f" % acc) if acc is not None else "")
    path = os.path.join(MODELS_DIR, name + ".h5")
    model.save(path)

In [7]:
train_set, test_set = split_train_dataset(pulsars, 0.2)

In [8]:
X_train, Y_train = train_set[:, :-1], train_set[:, -1]
X_test, Y_test = test_set[:, :-1], test_set[:, -1]

In [9]:
# Create Model
input_dimension = np.size(X_train, axis=1)

def create_model():
    model = Sequential()
    model.add(Dense(32, input_dim=input_dimension, activation='relu'))
    model.add(Dense(12, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [10]:
# Create and Compile Model
model = create_model()

In [11]:
# Fit the Model
model.fit(X_train, Y_train, epochs=100, batch_size=15)

Epoch 1/100
14319/14319 [==============================] - 4s 249us/step - loss: 0.1564 - acc: 0.9654
Epoch 2/100
14319/14319 [==============================] - 3s 198us/step - loss: 0.1200 - acc: 0.9707
Epoch 3/100
14319/14319 [==============================] - 3s 196us/step - loss: 0.1108 - acc: 0.9717
Epoch 4/100
14319/14319 [==============================] - 3s 196us/step - loss: 0.0993 - acc: 0.9739
Epoch 5/100
14319/14319 [==============================] - 3s 208us/step - loss: 0.0995 - acc: 0.9741
Epoch 6/100
14319/14319 [==============================] - 3s 198us/step - loss: 0.0962 - acc: 0.9735
Epoch 7/100
14319/14319 [==============================] - 3s 196us/step - loss: 0.0951 - acc: 0.9725
Epoch 8/100
14319/14319 [==============================] - 3s 195us/step - loss: 0.0923 - acc: 0.9740
Epoch 9/100
14319/14319 [==============================] - 3s 213us/step - loss: 0.0921 - acc: 0.9740
Epoch 10/100
14319/14319 [==============================] - 3s 201us/step - loss: 

Epoch 80/100
14319/14319 [==============================] - 3s 190us/step - loss: 0.0738 - acc: 0.9784
Epoch 81/100
14319/14319 [==============================] - 3s 189us/step - loss: 0.0738 - acc: 0.9780
Epoch 82/100
14319/14319 [==============================] - 3s 244us/step - loss: 0.0737 - acc: 0.9786
Epoch 83/100
14319/14319 [==============================] - 3s 210us/step - loss: 0.0742 - acc: 0.9790
Epoch 84/100
14319/14319 [==============================] - 3s 187us/step - loss: 0.0743 - acc: 0.9784
Epoch 85/100
14319/14319 [==============================] - 3s 189us/step - loss: 0.0724 - acc: 0.9788
Epoch 86/100
14319/14319 [==============================] - 3s 189us/step - loss: 0.0738 - acc: 0.9781
Epoch 87/100
14319/14319 [==============================] - 3s 189us/step - loss: 0.0742 - acc: 0.9779
Epoch 88/100
14319/14319 [==============================] - 3s 189us/step - loss: 0.0731 - acc: 0.9786
Epoch 89/100
14319/14319 [==============================] - 3s 191us/step

In [12]:
# Evaluate the Model
scores = model.evaluate(X_test, Y_test)

3579/3579 [==============================] - 0s 28us/step


In [13]:
print("Accuracy:", scores[1] * 100)

Accuracy: 98.1000279458


In [14]:
from sklearn.metrics import classification_report

# Classification metrics on test data
predictions = model.predict(X_test)
Y_test_predictions = [int(y + 0.5) for y in predictions]
print(classification_report(Y_test, Y_test_predictions))

             precision    recall  f1-score   support

        0.0       0.98      1.00      0.99      3259
        1.0       0.94      0.84      0.89       320

avg / total       0.98      0.98      0.98      3579



In [15]:
# Classification metrics on all data

predictions = model.predict(pulsars[:, :-1])
Y_test_predictions = [int(y + 0.5) for y in predictions]
print(classification_report(pulsars[:,-1].astype(int), Y_test_predictions))

             precision    recall  f1-score   support

          0       0.98      0.99      0.99     16259
          1       0.94      0.83      0.88      1639

avg / total       0.98      0.98      0.98     17898



In [16]:
save_model(model, 'pulsar', scores[1] * 100)